## Camada bronze (raw)

- Aqui irei ingerir os dados de forma bruta, apenas disponibilizando para ser processado nas demais camadas

### Ingestao

In [0]:
if not any(mount.mountPoint == '/mnt/datalayer' for mount in dbutils.fs.mounts()):    dbutils.fs.mount(
    source='wasbs://datalayer@dbdatalayer.blob.core.windows.net',
    mount_point='/mnt/datalayer',
    extra_configs={'fs.azure.account.key.dbdatalayer.blob.core.windows.net': dbutils.secrets.get(scope='data_connection', key='blobkey')}
)


In [0]:
df_channel = spark.read.format("csv").option("header", "true").load("/mnt/datalayer/raw/abi_bus_case1_beverage_channel_group_20210726.csv")

display(df_channel)

In [0]:
# Aparentemente a chave esta na coluna TRADE_CHNL_DESC
df_channel.groupBy("TRADE_CHNL_DESC").count().show()

In [0]:
#separador e encoding diferente do primeiro arquivo

df_sales = spark.read.format("csv").option("header", "true").option("sep", "\t").option("encoding","UTF-16").load("/mnt/datalayer/raw/abi_bus_case1_beverage_sales_20210726.csv")

display(df_sales)

### Escrita

- Como estamos na camada bronze, nao irei me preocupar muito com schema 

In [0]:
df_sales.withColumnRenamed('$ Volume', 'Volume').write.mode("overwrite").saveAsTable("ab_inbev.bronze.beverage_sales")

In [0]:
df_channel.write.mode("overwrite").saveAsTable("ab_inbev.bronze.beverage_channels")

#### Tabelas juntas (requirement #1)

In [0]:
df_merged = df_sales.join(df_channel, on="TRADE_CHNL_DESC", how="left")

display(df_merged)

In [0]:
#validacoes de sucesso no merge
df_merged.filter(df_merged.TRADE_GROUP_DESC.isNull()).count() == 0

In [0]:
#validacoes de sucesso no merge
df_merged.count() == df_sales.count()

In [0]:
df_merged.withColumnRenamed('$ Volume', 'Volume').write.mode("overwrite").saveAsTable("ab_inbev.bronze.beverage_sales_and_channels")